In [1]:
import numpy as np

FreqB = (4.5, 6.0)  # GHz
peakB = (0.5, 2.0)
widthB = (0.005, 0.01)  # GHz
noice = 0.25
fpts = np.linspace(FreqB[0], FreqB[1], 300)  # (h,)
transitions = [(0, 1), (0, 2), (1, 2), (0, 3), (1, 3)]
weights = np.array([2, 1, 0.5, 0.5, 0.2])

In [2]:
from scqubits import Fluxonium

def calculate_spectrum(flxs, EJ, EC, EL, evals_count=4, cutoff=50):
    fluxonium = Fluxonium(EJ, EC, EL, flux=0.0, cutoff=cutoff, truncated_dim=evals_count)
    spectrumData = fluxonium.get_spectrum_vs_paramvals(
        "flux", flxs, evals_count=evals_count
    )

    return spectrumData.energy_table

def lorfunc(x, yscale, x0, gamma):
    return yscale / (1 + ((x - x0) / gamma) ** 2)


def make_spectrum(self, energies):
    # energies: (n, m')
    fs = []
    yscales = []
    for idx in range(len(transitions)):
        i, j = transitions[idx]
        w = weights[idx]
        fs.append(energies[:, j] - energies[:, i])
        yscales.append(w * np.random.uniform(0.8, 1.2))
    fs = np.stack(fs, axis=1)  # (n, m)
    yscales = np.array(yscales)  # (m,)
    _, m = fs.shape

    # calculate the spectrum
    yscales *= np.random.choice([-1, 1], m)
    yscales = yscales[None, None, :]  # (1, 1, m)
    gammas = np.random.uniform(*widthB, (1, 1, m))
    xs = fpts[None, :, None]  # (1, h, 1)
    x0s = fs[:, None, :]  # (n, 1, m)
    ys = lorfunc(xs, yscales, x0s, gammas)  # (n, h, m)
    spectrum = np.sum(ys, axis=-1)  # (n, h)

    # add noise
    spectrum += np.random.normal(0, noice, spectrum.shape)

    return spectrum # (n, h)

In [ ]:
EJb = (4.0, 6.0)
ECb = (0.5, 1.0)
ELb = (1.0, 2.0)

level = 4
total_num = 121

flxs = np.linspace(0.0, 1.0, total_num)
EJ = np.random.uniform(*EJb)
EC = np.random.uniform(*ECb)
EL = np.random.uniform(*ELb)
print("EJ, EC, EL:", EJ, EC, EL)
energies = calculate_spectrum(flxs, EJ, EC, EL, level, 50)

In [4]:
spectrum = make_spectrum(flxs, energies)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.pcolormesh(flxs, fpts, spectrum.T)

# NN

In [ ]:
# fit the spectrumData
import torch
from modules.model import PredictNet
from modules.baseblock import ResNet18
from torch.nn import MSELoss

model_path = "ckpt/resnet18.pth"
model = PredictNet(ResNet18)
model.load_state_dict(torch.load(model_path, weights_only=True))
criterion = MSELoss()

model.eval()
model.cuda()
criterion.cuda()
with torch.no_grad():
    nn_params = model(torch.tensor(
        spectrum,
        dtype=torch.float32,
        device="cuda"
    )[None, ...])
    true_params = (
        (EJ - EJb[0]) / (EJb[1] - EJb[0]),
        (EC - ECb[0]) / (ECb[1] - ECb[0]),
        (EL - ELb[0]) / (ELb[1] - ELb[0]),
    )
    loss = criterion(nn_params, torch.tensor([true_params], dtype=torch.float32, device="cuda"))
    print("Loss:", loss.item())

    nn_params = nn_params.cpu().numpy()[0].tolist()
    nn_params = (
        nn_params[0] * (EJb[1] - EJb[0]) + EJb[0],
        nn_params[1] * (ECb[1] - ECb[0]) + ECb[0],
        nn_params[2] * (ELb[1] - ELb[0]) + ELb[0],
    )

# print the results
print("True params:", EJ, EC, EL)
print("Fitted params:", *nn_params)

In [ ]:
f_energies = calculate_spectrum(flxs, *nn_params, level, 50)

In [ ]:

for i, j in transitions:
    plt.plot(flxs, f_energies[:, j] - f_energies[:, i], label=f"{i}-{j}")

for i, j in transitions:
    plt.plot(flxs, energies[:, j] - energies[:, i], "--", label=f"{i}-{j}")

plt.ylim(FreqB)
plt.legend()
plt.show()

# Scipy

In [9]:
from scipy.ndimage import gaussian_filter1d

def NormalizeData(signals2D: np.ndarray, axis=None) -> np.ndarray:
    # normalize on given axis
    mins = np.nanmin(signals2D, axis=axis, keepdims=True)
    maxs = np.nanmax(signals2D, axis=axis, keepdims=True)
    meds = np.nanmedian(signals2D, axis=axis, keepdims=True)
    return (signals2D - meds) / (maxs - mins)


def spectrum_analyze(flxs, fpts, amps, ratio):
    """
    flxs: 1D array, flux points
    fpts: 1D array, frequency points
    amps: 2D array, shape: (len(fpts), len(flxs))
    """
    # use guassian filter to smooth the spectrum
    amps = gaussian_filter1d(amps, 3, axis=0)
    amps = NormalizeData(amps, 0)  # normalize on frequency axis
    amps = np.abs(amps)

    # plot max point and min point of each row
    max_ids = np.argmax(amps, axis=0)
    maxs = amps[max_ids, np.arange(amps.shape[1])]

    # max points
    max_masks = maxs >= ratio
    fs = fpts[max_ids]
    fs[~max_masks] = np.nan
    mask = ~np.isnan(fs)

    return flxs[mask], fs[mask]

s_flxs, s_fs = spectrum_analyze(flxs, fpts, spectrum.T, 0.6)

In [10]:
# remove some close points
mask = np.ones(len(s_flxs), dtype=bool)
t_d2 = np.sqrt((s_flxs[-1] - s_flxs[0])**2 + (s_fs[-1] - s_fs[0])**2)/20
prev = 0
for i in range(1, len(s_flxs)):
    d_flx = s_flxs[i] - s_flxs[prev]
    d_fs = s_fs[i] - s_fs[prev]
    d2 = np.sqrt(d_flx**2 + d_fs**2)
    if d2 < t_d2:
        mask[i] = False
    else:
        prev = i

s_flxs = s_flxs[mask]
s_fs = s_fs[mask]

In [ ]:
plt.figure()
plt.pcolormesh(flxs, fpts, spectrum.T)
plt.scatter(s_flxs, s_fs, c='r')
plt.show()

In [12]:
from scipy.optimize import minimize


def fit_spectrum(flxs, fpts, params, maxfun=10):
    """
    Fit the fluxonium spectrum to the experimental spectrum
    flxs: 1D array of flux values, shape (n,)
    fpts: 2D array of transition frequencies, shape (n,)
    """

    def loss_func(params):
        energies = calculate_spectrum(flxs, *params, 4, cutoff=50)

        # only fit 0-1, 0-2, 1-2, 0-3, 1-3
        fs = []
        for i, j in transitions:
            fs.append(energies[:, j] - energies[:, i])
        fs = np.stack(fs, axis=1) # (n, m)
        dist = np.abs(fpts[:, None] - fs) # (n, m)
        # loss_fs = np.nanmin(loss_fs, axis=1) # (n,)
        dist = dist / weights[None, :] # make some transitions more attractive
        loss_fs = np.nanmin(dist, axis=1) # (n,)

        return np.nansum(loss_fs)

    res = minimize(
        loss_func,
        params,
        bounds=(EJb, ECb, ELb),
        method="L-BFGS-B",
        options={"maxfun": maxfun},
    )

    return res.x

In [ ]:
# fit the spectrumData
sp_params = fit_spectrum(s_flxs, s_fs, params=nn_params, maxfun=50)

# print the results
print("True params:", EJ, EC, EL)
print("Fitted params:", *sp_params)

In [ ]:
f_energies = calculate_spectrum(flxs, *sp_params, level, 50)

In [ ]:
for i, j in transitions:
    plt.plot(flxs, f_energies[:, j] - f_energies[:, i], label=f"{i}-{j}")

# plot the true values in dashed lines
for i, j in transitions:
    plt.plot(flxs, energies[:, j] - energies[:, i], "--", label=f"{i}-{j}")

plt.scatter(s_flxs, s_fs, color="red")
plt.ylim(FreqB)
plt.legend()
plt.show()